<a href="https://colab.research.google.com/github/azmeraw-tech/Spee-checker-For-Amharic-Languages/blob/main/amharic_spell_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install amseg
!pip install num2words
!pip install pytesseract
!pip install pdf2image

In [ ]:
import re
import string
import sys
import re
import math
import nltk
import amseg
from collections import Counter
from amseg.amharicSegmenter import AmharicSegmenter
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import *
from tensorflow.python.client import device_lib
import numpy as np
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()
#### Do this once
nltk.download('punkt')
import sys
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
source = sys.argv[1]
result = source+".txt"
def normalize_char_level_missmatch(input_token):
        rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
        rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
        rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
        rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
        rep5=re.sub('[ሕኅ]','ህ',rep4)
        rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
        rep7=re.sub('[ሠ]','ሰ',rep6)
        rep8=re.sub('[ሡ]','ሱ',rep7)
        rep9=re.sub('[ሢ]','ሲ',rep8)
        rep10=re.sub('[ሣ]','ሳ',rep9)
        rep11=re.sub('[ሤ]','ሴ',rep10)
        rep12=re.sub('[ሥ]','ስ',rep11)
        rep13=re.sub('[ሦ]','ሶ',rep12)
        rep14=re.sub('[ዓኣዐ]','አ',rep13)
        rep15=re.sub('[ዑ]','ኡ',rep14)
        rep16=re.sub('[ዒ]','ኢ',rep15)
        rep17=re.sub('[ዔ]','ኤ',rep16)
        rep18=re.sub('[ዕ]','እ',rep17)
        rep19=re.sub('[ዖ]','ኦ',rep18)
        rep20=re.sub('[ጸ]','ፀ',rep19)
        rep21=re.sub('[ጹ]','ፁ',rep20)
        rep22=re.sub('[ጺ]','ፂ',rep21)
        rep23=re.sub('[ጻ]','ፃ',rep22)
        rep24=re.sub('[ጼ]','ፄ',rep23)
        rep25=re.sub('[ጽ]','ፅ',rep24)
        rep26=re.sub('[ጾ]','ፆ',rep25)
        #Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል
        rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
        rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
        rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
        rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
        rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
        rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
        rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
        rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
        rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
        rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
        rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
        rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
        rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
        rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
        rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
        rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
        rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
        rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
        rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
        rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
        rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
        rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ

        return rep48

In [ ]:
def normalize_char_level_missmatch(input_token):
        rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
        rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
        rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
        rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
        rep5=re.sub('[ሕኅ]','ህ',rep4)
        rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
        rep7=re.sub('[ሠ]','ሰ',rep6)
        rep8=re.sub('[ሡ]','ሱ',rep7)
        rep9=re.sub('[ሢ]','ሲ',rep8)
        rep10=re.sub('[ሣ]','ሳ',rep9)
        rep11=re.sub('[ሤ]','ሴ',rep10)
        rep12=re.sub('[ሥ]','ስ',rep11)
        rep13=re.sub('[ሦ]','ሶ',rep12)
        rep14=re.sub('[ዓኣዐ]','አ',rep13)
        rep15=re.sub('[ዑ]','ኡ',rep14)
        rep16=re.sub('[ዒ]','ኢ',rep15)
        rep17=re.sub('[ዔ]','ኤ',rep16)
        rep18=re.sub('[ዕ]','እ',rep17)
        rep19=re.sub('[ዖ]','ኦ',rep18)
        rep20=re.sub('[ጸ]','ፀ',rep19)
        rep21=re.sub('[ጹ]','ፁ',rep20)
        rep22=re.sub('[ጺ]','ፂ',rep21)
        rep23=re.sub('[ጻ]','ፃ',rep22)
        rep24=re.sub('[ጼ]','ፄ',rep23)
        rep25=re.sub('[ጽ]','ፅ',rep24)
        rep26=re.sub('[ጾ]','ፆ',rep25)
        #Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል
        rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
        rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
        rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
        rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
        rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
        rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
        rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
        rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
        rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
        rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
        rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
        rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
        rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
        rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
        rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
        rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
        rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
        rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
        rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
        rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
        rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
        rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ

        return rep48

In [ ]:
#def abbreviations(data):
short_form_dict= {
        'ሆ/ል'	:'ሆስፒታል',
        'መ/ቅ' :	'መጸሐፍ ቅዱስ',
        'መ/ቤት' : 'መሥሪያ ቤት',
        'ሚ/ሩ'	: 'ሚኒስትሩ',
        'ም/':	'ምክትል',
        'ም/ቤት':	'ምክር ቤት',
        'ሠ/ፌዴሬሽን'	: 'ሠራተኛ ፌዴሬሽን',
        'ሻ/': 	'ሻምበል',
        'ቤ/መ': 	'ቤተ መንግስት',
        'ቤ/ክ' :'ቤተ ክርስትያን',
        'ተ/' :'ተክለ',
        'ኃ/' :'ኃይለ',
        'አ/አ' :	'አዲስ አባባ',
        'ኮ/ል' :	'ኮለኔል',
        'ወ/' :'ወልደ',
        'ወ/ሪት': 'ወይዘሪት',
        'ወ/ሮ' :	'ወይዘሮ',
        'ዓ/ም':	'ዓመተ ምህረት',
         'ዓ.ም': 'ዓመተ ምህረት',
        'ዓ/ዓ' :	'ዓመተ ዓለም',
        'ዶ/ር' :	'ዶክተር',
        'ጄ/ል' :	'ጄኔራል',
        'ገ/' : 'ገብረ',
        'ጠ/ሚ': 'ጠቅላይ ሚኒስትር',
        'ጠ/ሚ/ቢሮ' : 'ጠቅላይ ሚኒስትር ቢሮ',
        'ጠ/ሚኒስትር':	'ጠቅላይ ሚኒስትር',
        'ጽ/ቤት' : 'ጽህፈት ቤት',
        'ፍ/ቤት': 'ፍርድ ቤት',
        'ፕ/ር'	:'ፕሮፌሰር',
        'ፕ/ት'	:'ፕሬዚዳንት',
        'ት/ቤት' :	'ትምህርት ቤት'
    }
#text = "ዓ.ም"

#def expand_short_form(input_short_word):
 # if input_short_word in short_form_dict:
  #  return short_form_dict[input_short_word]
  #else:
   # return input_short_word

#test=expand_short_form(text)
#print(test)

In [ ]:
def expand_word(word, short_form_dict):
    if word in short_form_dict:
        return short_form_dict[word]
    else:
        return word
def expand_sentence(sentence, short_form_dict):
    # Split the sentence into words
    words = sentence.split()
    # Expand each word in the sentence
    expanded_words = [expand_word(word, short_form_dict) for word in words]
    # Join the expanded words back into a sentence
    expanded_sentence = " ".join(expanded_words)
    return expanded_sentence


In [ ]:
import num2words

def create_amharic_dict(start_num, end_num):
    amharic_dict = {}
    for num in range(start_num, end_num+1):
        amharic_word = num2words.num2words(num, lang='am')
        amharic_dict[num] = f'{amharic_word}'
    return amharic_dict

# Define the range of numbers to convert
start_num = 1
end_num = 1000

# Create the Amharic dictionary
#amharic_dict = create_amharic_dict(start_num, end_num)
#print(amharic_dict)

def num_to_amharic(num):
    """
    This function takes a number as input and returns its Amharic word representation.
    """
    amharic_dict = create_amharic_dict(start_num, end_num)


    if num in amharic_dict:
        return amharic_dict[num]
    else:
        return str(num)

def replace_numbers_with_amharic(text):
    """
    This function takes a string as input and replaces any numbers in it with their
    Amharic word representation.
    """
    words = text.split()
    result = []
    for word in words:
        if word.isdigit():
            result.append(num_to_amharic(int(word)))
        else:
            result.append(word)
    return ' '.join(result)


In [ ]:
# Remove the rest of punctuation marks
def preprocessing(text):
    punctuations =  '''`÷×؛<>_()*&^%][ـ،:"؟.,'{}~¦+|!”…“–ـ/$£•●\፡\።\?\!\!\@\#\$\%\^\«\»\&\*\(\)\…\[\]\{\}\;\“\”\›\’\‘\"\'\:\,\.\‹\/\<\>\?\\\\|\`\´\~\-\=\+\፡\።\፤\;\፦\፥\፧\፨\፠\፣]'''
    punctuation = string.punctuation
    #remove geez characters
    unwanted_chara = "�¿áóóó□"
    remove = punctuations + punctuation +  unwanted_chara

    replace_slash = str.maketrans('/', ' ', '')
    text = text.translate(replace_slash)
    remove_punc = str.maketrans('', '', remove)
    text = text.translate(remove_punc)
    english_characters = re.compile(r'[a-zA-Z]')
    text = re.sub(english_characters, '', text)
    text=re.sub(r'\w*\d\w*', '', text).strip() # remove number and words which contain eng
    text=re.sub(r'\b\w{9,}\b','',text) # remove words which contain more than 9 characters

    return text


In [ ]:
path='/content/corpusTest.txt'
with open(path,'r', errors="ignore") as f:
  text=f.read()
sent_punct = ['።', '!', '?', '፡፡']
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)
text = segmenter.tokenize_sentence(text)
text = list(set(text))
print(text[:4])
#print(text)

['በእንስሳት ሊገኝ የሚችል ጥቅምን ከሚያሳጡ ምክንያቶች ይሄ አንዱ በመሆን ይጠቀሳል ሲሉ ያብራራሉ።', 'በልብ።', 'የነዳጅ የመሸጫ ዋጋ ማስተካከያ ይፋ ከተደረገበት።', 'ትላልቅ ፕሮጀክቶች ካልተተገበሩ በስተቀር በመንግሥት በጀት ብቻ የሥራ አጥነትን ችግር መቅረፍ አይቻልም፡፡']


In [ ]:
with open(result, "w+") as DATA:
    for segment in text:
        segment = segment.strip()
        #segment = remove_diacritics(segment)
        #segment=expand_short_form(segment)
        segment = preprocessing(segment)
        segment = normalize_char_level_missmatch(segment)
        segment = segment.strip()
        segment = " ".join(segment.split())                   # remove extra white-spaces
        segment = " ".join(segment.split())              # trancate to 15 tokens
        if segment != "" and len(segment.split())>2:          # not empty and > 3 tokens (one is <s>)
            segment = "<s> " + segment
            #print(len(seg))
                                 # adding a start token
            DATA.write(segment + "\n")
            print(segment)
            #if segment[4] in text:
                #clean.write(segment + "\n")

Streaming output truncated to the last 5000 lines.
<s> እንደመጡ መለየት እንደሚቻል
<s> ያሰርዋቸውን ነጮች ለማስለቀቅ
<s> ሌሎች ደግሞ ልብስ ከገዙ በኋላ የሰውነት ቅርፅ ለውጥ ይከሰታል
<s> በቴክኖሎጂ መጥቆዋል ተብሎ በሚለፈፍበት ዛሬ ላይ ቆመን ቀን ካልሆነ ብርሀን የማያይ በፋኖስ ካልሆነ ድግዝግዙን ጭለማ የመግፈፍ
<s> አምረውና ተውበው ለመታየት ደግሞ ማህበረሰቡ ያመነበትን ወይም የተቀበለውን የመዋቢያ አይነቶች ይጠቀማሉ
<s> እንደ እሳቸው መልስ ከሆነ የዘንድሮው የአገራችን የዝናብ መጠን መደበኛ በአንዳንድ አካባቢዎች ደግሞ ከመደበኛም በላይ ነው
<s> በዚህ አመት ተገንብቶ የተጠናቀቀው የድሬዳዋ ደወሌ ጂቡቲ የፍጥነት መንገድ ሲሆን ይህ ድሬዳዋን ከጅቡቲ ጋር የሚያገናኝ ፈጣን
<s> የተባለው እንዲፈፀም ጋዜጣውን ለማሳተም ሂደቱን ጀመሩ
<s> ሰማይ ምድሩ ንፁህ ሆኖ ነው የሚታይ
<s> የአንድ የግሪክ ሀብታም ቤት መላውን የባሪያ አሳዳሪ ኢኮኖሚ ማለትም መላውን ያገር ኢኮኖሚ በአነስተኛ መልኩ የሚወክል ነበር
<s> የተለያዩ የዘርፉ ምሁራንም የአየር ንብረት ተፅእኖ አለማችን ውድ ዋጋ በቀጣይ ጨምሮ በማስረዳት ላይ ተጠምዷል
<s> የኢትዮጵያ የብረታ ብረት ኢንዱስትሪ ከአመት አመት ከፍተኛ የሚባል እድገትን እያስመዘገበ ለበርካታ ዜጎች የስራ እድል በመፍጠር በአገሪቱ የልማት እንቅስቃሴ መጎልበትና የምጣኔ ሀብት እድገት ከፍተኛ ሚናን እየተጫወተ ይገኛል
<s> በተያያዘ የሚስተዋሉ ችግሮችን
<s> ስድስት አመት በኋላ ደግሞ ዳግም በፔሊካን ትሬዲንግ የግል ድርጅት ውስጥ በመግባት በነበረ ቦታቸው መስራት ጀመሩ
<s> በቅርቡ ገበያው ይሰፋል
<s> በተለይ የሴትነት ሀላፊነት ሲደመርበት ጫናው ይበዛል
<s> በተለይም ተግባብቶ ገንዘብ ማምጣት

In [ ]:
# Pre-process data
data_file = "/content/-f.txt"                     # Dataset file as an argument
data = open(data_file).read()
corpus=data.split("\n")
#print(corpus[0])

In [ ]:
vocab_size = 1000
out_of_vocab = "<unk>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=out_of_vocab)
tokenizer.fit_on_texts(corpus)

In [ ]:
# Create input sequences using list of tokens
input_sequences = []            # words before the current word/label
input_sequences_reversed = []   # words after the current word/label
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        n_gram_sequence_rev = token_list[i:]
        n_gram_sequence_rev.reverse()
        input_sequences_reversed.append(n_gram_sequence_rev)

In [ ]:
# Pad sequences
max_sequence_len = 100
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences_reversed = np.array(pad_sequences(input_sequences_reversed, maxlen=max_sequence_len, padding='pre'))
print(input_sequences[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1]


In [ ]:
# Create predictors (words before and after) and label (current word)
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
predictors_rev, label_rev = input_sequences_reversed[:,:-1], input_sequences_reversed[:,-1]


In [ ]:
# Input
left_input = Input(shape=(max_sequence_len-1,), name="left_in")
right_input = Input(shape=(max_sequence_len-1,),  name="right_in")

# Embedding
left_features = Embedding(vocab_size, 256)(left_input)
right_features = Embedding(vocab_size, 256)(right_input)

# Bidirectional LSTM
left_features = Bidirectional(LSTM(512, return_sequences = True))(left_features)
right_features = Bidirectional(LSTM(512, return_sequences = True))(right_features)

# LSTM
left_features = LSTM(128)(left_features)
right_features = LSTM(128)(right_features)

# Merge all available features into a single large vector via concatenation
concat = concatenate([left_features, right_features], name='concatenate')

# Dense - output
pred = Dense(vocab_size, activation='softmax', name="pred")(concat)



In [ ]:
# Instantiate an end-to-end model predicting the next word based on both the left and right inputs
model = Model(
    inputs = [left_input, right_input],
    outputs = pred
)


model.compile(
    optimizer = Adam(),
    loss = SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_in (InputLayer)           [(None, 99)]         0           []                               
                                                                                                  
 right_in (InputLayer)          [(None, 99)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 99, 256)      256000      ['left_in[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 99, 256)      256000      ['right_in[0][0]']               
                                                                                            

In [ ]:
# Define callbacks
my_callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=3),
    ModelCheckpoint(filepath='model-dual-input/model.{epoch:02d}-{accuracy:.3f}-{val_accuracy:.3f}', verbose=1)
]


In [ ]:
# Train the model
model.fit(
    {"left_in": predictors, "right_in": predictors_rev},
    {'pred': label},
    epochs=1,
    batch_size=128,
    validation_split=0.02,
    callbacks=my_callbacks
)

26203/26203 [==============================] - ETA: 0s - loss: 2.9029 - accuracy: 0.5693
Epoch 1: saving model to model-dual-input/model.01-0.569-0.574


26203/26203 [==============================] - 5367s 204ms/step - loss: 2.9029 - accuracy: 0.5693 - val_loss: 2.7250 - val_accuracy: 0.5736


In [ ]:
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()
import tkinter as tk
from tkinter import messagebox
#### Do this once
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from tensorflow.keras.models import load_model
print("Loading the spelling the checker model...")
model = load_model("/content/drive/MyDrive/saved_model.h5")

In [ ]:
model.save('/content/saved_model.h5')

In [ ]:
def generate_ngrams(text):
    text = "<s>" + text

    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]

    all_grams = []

    for n in range(2, len(tokens)+1):
        grams = [w for w in nltk.ngrams(tokens, n=n)][0]

        grams_rev = tokens[n:]
        grams_rev.reverse()
        all_grams.append((list(grams), list(grams_rev)))

    return all_grams
#data=generate_ngrams(text)


In [ ]:
while True:
    text_to_correct = input("Enter text to correct (or press Enter to exit): ")

    if not text_to_correct:
        break

    print("Currently correcting:", text_to_correct)

    ngrams = generate_ngrams(text_to_correct)
    correct = None
    suggestions = []

    for ngram in ngrams:

        if len(ngram[0]) > 2 and correct != 1 and len(suggestions) != 0:
            seed_text_ltr = " ".join(word for word in ngram[0][:-2]) + " " + suggestions[0][2]
        else:
            seed_text_ltr = " ".join(word for word in ngram[0][:-1])

        current_word = ngram[0][-1]
        seed_text_rtl = " ".join(word for word in ngram[1])
        print(seed_text_ltr, "*_", current_word, "_*", seed_text_rtl)

        token_list = tokenizer.texts_to_sequences([seed_text_ltr])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        token_list_rev = tokenizer.texts_to_sequences([seed_text_rtl])[0]
        token_list_rev = pad_sequences([token_list_rev], maxlen=max_sequence_len-1, padding='pre')

        predicted_id = np.argmax(model.predict([token_list, token_list_rev]), axis=-1)
        predicted_word = tokenizer.sequences_to_texts([predicted_id])[0]
        print(predicted_word)


        predicted_probs = model.predict([token_list, token_list_rev])
        predicted_best = np.argsort(-predicted_probs, axis=-1)[0][:4500]

        suggestions = []
        correct = None

        for prob in predicted_best:
            output_word = tokenizer.sequences_to_texts([[prob]])[0]
            ed = nltk.edit_distance(current_word, output_word)

            if ed ==0:#ed=edit_distance
                print("I got this one; it seems correct -->", current_word, "=", output_word)
                correct = 1
                break
            elif len(current_word)<=3 and ed ==1:
                suggestions.append((ed, current_word, output_word))
            elif len(current_word)>3 and ed <=2:
                suggestions.append((ed, current_word, output_word))
            else:
                continue

        if correct == 2:
            print("Not sure")
        elif correct == 1:
            print("CORRECT")
        elif correct != 1 and len(suggestions) > 0:
            correct = 0
            print("WRONG")
        elif correct != 1 and len(suggestions) == 0:
            print("I do not know!")

        if len(suggestions) > 0:
            for suggest in suggestions:
                lemmas_cw = []
                lemmas_cw.append(suggest[1])
                lemmas_cw.append(st.suf1(suggest[1]))
                lemmas_cw.append(st.suf32(suggest[1]))
                lemmas_cw.append(st.pre1(suggest[1]))
                lemmas_cw.append(st.pre32(suggest[1]))

                lemmas_ow = []
                lemmas_ow.append(suggest[2])
                lemmas_ow.append(st.suf1(suggest[2]))
                lemmas_ow.append(st.suf32(suggest[2]))
                lemmas_ow.append(st.pre1(suggest[2]))
                lemmas_ow.append(st.pre32(suggest[2]))

                if correct != 1 and len(suggest[1]) > 7:
                    for l in lemmas_cw:
                        if l in lemmas_ow:
                            correct = 2
                            print("I got the lemma; it seems correct -->", current_word, "~", suggest[2])


        print("Suggestions:", " - ".join([suggest[2] for suggest in suggestions]))

